In [183]:
import requests
import json
from dataclasses import dataclass, fields
import itertools

%load_ext autoreload
%autoreload 2

from api import ESPN_API
from classes import Stat, Schedule, Matchup, Week, Player
from constants import *
espn_api = ESPN_API()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [185]:
!pip install numpy

     |████████████████████████████████| 15.6 MB 2.5 MB/s eta 0:00:01     |█████████████████████████       | 12.2 MB 2.0 MB/s eta 0:00:02


In [188]:
import numpy as np

In [115]:
player_rating_dict = {
    'Zach LaVine': 15.55,
    'Bradley Beal': 10.75,
    'Kawhi Leonard': 11.79,
    'Nikola Vucevic': 13.77,
    'Larry Nance Jr.': 7.51,
    'Luka Doncic': 11.37,
    'Mason Plumlee': 5.45,
    'Clint Capela': 4.08,
    'Richaun Holmes': 10.03,
    'Keldon Johnson': 6.88,
    'P.J. Washington': 6.55,
    "OG Anunoby": 6.50,
    'Nikola Jokic': 19.37,
    'Kristaps Porzingis': -2.68
}

In [167]:
def player_games(team_no, no_games, rating_dict):
    remaining_games = espn_api.remaining_games
    all_players = espn_api.players
    
    roster = espn_api.teams[team_no].roster
    players = list(roster.keys())
    
    games_played_dict = {}
    for player in players:
        games_played_dict[player] = {}
        games_played_dict[player]['Rating'] = rating_dict[player]
    games_played_dict = dict(sorted(games_played_dict.items(), key = lambda item: item[1]['Rating'], reverse=True))
    
    for player in games_played_dict.keys():
        games_left = remaining_games[all_players[player].nba_tid]
        if no_games >= games_left:
            games_played_dict[player]['Games'] = games_left
            no_games -= games_left
        else:
            games_played_dict[player]['Games'] = no_games
            no_games = 0

    return games_played_dict 
    

In [168]:
player_games(6, 30, player_rating_dict)

{'Zach LaVine': {'Rating': 15.55, 'Games': 2},
 'Nikola Vucevic': {'Rating': 13.77, 'Games': 3},
 'Kawhi Leonard': {'Rating': 11.79, 'Games': 3},
 'Luka Doncic': {'Rating': 11.37, 'Games': 3},
 'Bradley Beal': {'Rating': 10.75, 'Games': 3},
 'Richaun Holmes': {'Rating': 10.03, 'Games': 3},
 'Larry Nance Jr.': {'Rating': 7.51, 'Games': 3},
 'Keldon Johnson': {'Rating': 6.88, 'Games': 3},
 'P.J. Washington': {'Rating': 6.55, 'Games': 3},
 'OG Anunoby': {'Rating': 6.5, 'Games': 3},
 'Mason Plumlee': {'Rating': 5.45, 'Games': 1},
 'Clint Capela': {'Rating': 4.08, 'Games': 0},
 'Kristaps Porzingis': {'Rating': -2.68, 'Games': 0}}

In [170]:
cats = ['points', 'turnovers', 'fgm', 'fga', 'ftm', 'fta', 'made_threes', 
       'rebounds', 'assists', 'steals', 'blocks']

In [175]:
def overall_stats(games_dict, proj_type, cats):
    all_players = espn_api.players
    
    stats = {}
    for cat in cats:
        stats[cat] = 0
        for player in list(games_dict.keys()):
            stats[cat] += getattr(getattr(all_players[player].stats, proj_type).avg, cat) * games_dict[player]['Games']
    
    return stats

In [176]:
overall_stats(player_games(6,30, player_rating_dict), 's21', cats)

{'points': 585.1285714285715,
 'turnovers': 67.29835164835166,
 'fgm': 214.2945054945055,
 'fga': 432.2417582417583,
 'ftm': 103.96043956043957,
 'fta': 129.22252747252747,
 'made_threes': 52.57912087912087,
 'rebounds': 212.74615384615385,
 'assists': 122.69505494505493,
 'steals': 43.02967032967033,
 'blocks': 23.78956043956044}

In [177]:
def top_200_min():
    all_players = espn_api.players
    player_mins = {}
    for player in list(all_players.keys()):
        has_played = all_players[player].stats.s21
        if has_played:
            player_mins[player] = has_played.total.minutes
    sorted_player_mins = {player: mins for player,mins in sorted(player_mins.items(), key = lambda item: item[1], reverse=True)}
    sorted_player_mins = dict(itertools.islice(sorted_player_mins.items(), 200))
    return sorted_player_mins

In [178]:
players_min = top_200_min()
players_min

{'Julius Randle': 556.0,
 'RJ Barrett': 554.0,
 'Damian Lillard': 507.0,
 'Zach LaVine': 498.0,
 'Harrison Barnes': 492.0,
 'Domantas Sabonis': 488.0,
 'Coby White': 485.0,
 'LeBron James': 482.0,
 'Buddy Hield': 478.0,
 'Malcolm Brogdon': 477.0,
 'Jerami Grant': 471.0,
 "Devonte' Graham": 468.0,
 'Fred VanVleet': 465.0,
 'Dennis Schroder': 465.0,
 'Larry Nance Jr.': 464.0,
 'Terry Rozier': 463.0,
 'Nikola Jokic': 462.0,
 'Dejounte Murray': 459.0,
 'Khris Middleton': 456.0,
 'OG Anunoby': 451.0,
 'Paul George': 450.0,
 'Jrue Holiday': 450.0,
 'Gordon Hayward': 450.0,
 'Stephen Curry': 449.0,
 'Mitchell Robinson': 449.0,
 'Nikola Vucevic': 444.0,
 'Joe Harris': 444.0,
 'Keldon Johnson': 442.0,
 'CJ McCollum': 441.0,
 'Elfrid Payton': 436.0,
 'Donovan Mitchell': 435.0,
 'Kyle Lowry': 435.0,
 'Jamal Murray': 429.0,
 "De'Aaron Fox": 429.0,
 'Pascal Siakam': 427.0,
 'Devin Booker': 425.0,
 'Andrew Wiggins': 425.0,
 'Trae Young': 424.0,
 'Robert Covington': 422.0,
 'Anthony Davis': 420.0,
 '

In [109]:
all_players['James Harden'].stats.s21.total.games_played

10.0

In [192]:
def z_scores(player_mins, cats, proj_type):
    all_players = espn_api.players
    
    stats = {}
    stats['avg'] = {}
    stats['std'] = {}
    for cat in cats:
        cat_lst = []
        for player in list(player_mins.keys()):
            cat_lst.append(getattr(getattr(all_players[player].stats, proj_type).avg, cat))
        stats['avg'][cat] = numpy.mean(cat_lst)
        stats['std'][cat] = numpy.std(cat_lst)
    
    return stats

In [193]:
z_scores(players_min, cats, 's21')

{'avg': {'points': 13.94193503718504,
  'turnovers': 1.7302706598956599,
  'fgm': 5.100479353979354,
  'fga': 10.93881732156732,
  'ftm': 2.16816428016428,
  'fta': 2.8174647713397714,
  'made_threes': 1.572812049062049,
  'rebounds': 5.229507728382727,
  'assists': 3.103836025086025,
  'steals': 0.9122791236541238,
  'blocks': 0.5765793789543789},
 'std': {'points': 6.169284352260523,
  'turnovers': 0.9678365763977033,
  'fgm': 2.192651598067952,
  'fga': 4.4666461553091645,
  'ftm': 1.6318223133060483,
  'fta': 1.965931772784582,
  'made_threes': 0.9774428362482682,
  'rebounds': 2.6359266006284567,
  'assists': 2.142285625715789,
  'steals': 0.40782499597059596,
  'blocks': 0.5290533010501444}}